# Orchestrator-Workers Workflow

In [1]:
@file:DependsOn("dev.langchain4j:langchain4j:1.0.0-beta2")
@file:DependsOn("dev.langchain4j:langchain4j-anthropic:1.0.0-beta2")

In [2]:
import com.fasterxml.jackson.annotation.JsonCreator
import com.fasterxml.jackson.annotation.JsonProperty

/**
 * Represents a subtask assigned by the orchestrator that needs to be executed by the performer.
 */
data class Task @JsonCreator constructor(
    @JsonProperty("type") val type: String,
    @JsonProperty("description") val description: String
)

/**
 * The orchestra's response, containing a task analysis and breakdown into subtasks.
 */
data class OrchestratorResponse @JsonCreator constructor(
    @JsonProperty("analysis") val analysis: String,
    @JsonProperty("tasks") val tasks: List<Task>
)

data class WorkerResponse(val type: String, val description: String, val result: String) {
    override fun toString(): String {
        return """
            {
              "type": "$type",
              "description": "$description",
              "result": "$result"
            }
        """.trimIndent()
    }
}

In [3]:
interface LlmService {
    fun orchestrate(input: String): OrchestratorResponse
    fun perform(input: String): String
}

In [4]:
import kotlinx.serialization.Serializable

/**
 * Break down tasks and run them in parallel using worker LLMs
 */
class Orchestrator(val llm: LlmService) {

    companion object {
        val orchestratorPrompt: (String) -> String = {
            """
            Analyze this task and break it down into 2-3 distinct approaches:

            Task: $it

            Return your response in this JSON format:
            {
              "analysis": "Explain your understanding of the task and which variations would be valuable. Focus on how each approach serves different aspects of the task.",
              "tasks": [
                {
                  "type": "formal",
                  "description": "Write a precise, technical version that emphasizes specifications"
                },
                {
                  "type": "conversational",
                  "description": "Write an engaging, friendly version that connects with readers"
                }
              ]
            }
            """.trimIndent()
        }

        @Serializable
        data class Task(val type: String, val description: String)

        val workerPrompt: (String, String, String) -> String =
            { originalTask, taskType, taskDescription ->
                """
                Generate content based on:
                Task: $originalTask
                Style: $taskType
                Guidelines: $taskDescription
                """.trimIndent()
            }
    }

    /**
     * Process task by brealing it down and running subtasks in parallel.
     */
    fun process(taskInput: String): Pair<String, List<WorkerResponse>> {
        val orchestratorInput = orchestratorPrompt(taskInput)
        val orchestratorResponse = llm.orchestrate(orchestratorInput)

        val (analysis, tasks) = orchestratorResponse

        println(
            """
            === ORCHESTRATOR OUTPUT ===
            ANALYSIS: ${orchestratorResponse.analysis}

            TASKS: ${orchestratorResponse.tasks}
            """.trimIndent()
        )

        val workerResults = tasks.map { task ->
            val workerInput = workerPrompt(taskInput, task.type, task.description)
            val workerResponse = llm.perform(workerInput)


            println(
                """
                === WORKER RESULT (${task.type}) ===
                $workerResponse
                """.trimIndent()
            )

            WorkerResponse(task.type, task.description, workerResponse)
        }

        return analysis to workerResults
    }
}

In [5]:
import dev.langchain4j.model.anthropic.AnthropicChatModel
import dev.langchain4j.model.anthropic.AnthropicChatModelName

val model = AnthropicChatModel.builder()
    .apiKey(System.getenv("ANTHROPIC_API_KEY"))
    .modelName(AnthropicChatModelName.CLAUDE_3_7_SONNET_20250219)
    .maxTokens(4096)
    .temperature(0.1)
    .build()

In [6]:
import dev.langchain4j.service.AiServices

val service = AiServices.create(LlmService::class.java, model)

val orchestrator = Orchestrator(service)
val result = orchestrator.process("Write a product description for a new eco-friendly water bottle")

=== ORCHESTRATOR OUTPUT ===
ANALYSIS: This task requires creating a product description for an eco-friendly water bottle. The key challenge is to effectively communicate both the functional benefits (durability, capacity, materials) and emotional benefits (environmental impact, lifestyle alignment) of the product. Different approaches would be valuable to address various customer segments and marketing contexts. Some customers may prioritize technical specifications and environmental credentials, while others might be more motivated by lifestyle benefits and emotional appeals. The marketing context could also vary from e-commerce platforms to physical retail displays to social media content.

TASKS: [Task(type=technical-environmental, description=Create a specification-focused description that highlights the bottle's eco-friendly materials, technical features, and environmental impact metrics with precise data points), Task(type=lifestyle-benefits, description=Develop a benefits-orient